In [8]:
%reset
## 2015 He's 초기법


# 초기화와 (w,b) Activation function, dropout
# Xavier 초기법
# He's 초기법
# relu > Deep Learning 에서는 sigmoid를 사용하지 않아요
# 대신 Relu를 이용해요!!


import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler # Normalization
from sklearn.model_selection import train_test_split # train, test 데이터분리
from sklearn.model_selection import KFold # cross validation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

tf.reset_default_graph()

df = pd.read_csv('/Users/admin/Downloads/Digit_Recognizer_train.csv')

display(df)
# 결측치 확인
print(df.isnull().sum())


x_data = df.iloc[:,1:]
t_data = df['label']

# Data split 
from sklearn.model_selection import train_test_split
x_data_train, x_data_test, t_data_train, t_data_test=\
train_test_split(x_data,t_data,test_size = 0.3, random_state = 0)

# 데이터 정규화 (Normalization)
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
x_scaler.fit(x_data_train)
x_data_train_norm = x_scaler.transform(x_data_train)
x_data_test_norm = x_scaler.transform(x_data_test)

sess = tf.Session()   # Tensorflow node를 실행하기 위해서 session을 생성

# depth는 label의 종류 개수
t_data_train_onehot = sess.run(tf.one_hot(t_data_train, depth=10))  
t_data_test_onehot = sess.run(tf.one_hot(t_data_test, depth=10))


# Placeholder
X = tf.placeholder(shape = [None, 784], dtype = tf.float32)
T = tf.placeholder(shape = [None, 10], dtype = tf.float32)
dropout_rate = tf.placeholder(dtype = tf.float32)
#### 여기서부터 달라져요

# Weight & bias

W2 = tf.get_variable('weight2',shape=[784,256],
                     initializer = tf.contrib.layers.variance_scaling_initializer())
b2 = tf.Variable(tf.random.normal([256]), name='bias2')
_layer2 = tf.nn.relu(tf.matmul(X, W2) + b2)
layer2 = tf.nn.dropout(_layer2, rate = dropout_rate)


W3 = tf.get_variable('weight3',shape=[256,128], 
                     initializer = tf.contrib.layers.variance_scaling_initializer())
b3 = tf.Variable(tf.random.normal([128]), name='bias3')
_layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)
layer3 = tf.nn.dropout(_layer3, rate=dropout_rate)

W4 = tf.get_variable('weight4',shape=[128,10], 
                     initializer = tf.contrib.layers.variance_scaling_initializer())
b4 = tf.Variable(tf.random.normal([10]), name='bias4')


# Hypothesis
logit = tf.matmul(layer3, W4) + b4
H = tf.nn.softmax(logit)     # Multinomial Hypothesis

# Loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                 labels=T))
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)



num_of_epoch = 100
batch_size = 100


# 7. 학습진행
def run_train(sess,train_x, train_t):
    print('### 학습 시작 ###')
    sess.run(tf.global_variables_initializer())  # tf.Variable 초기화(W,b)

    total_batch = int(train_x.shape[0] / batch_size)
    for step in range(num_of_epoch):

        for i in range(total_batch):
            batch_x = train_x[i*batch_size:(i+1)*batch_size]
            batch_t = train_t[i*batch_size:(i+1)*batch_size]
            _, loss_val = sess.run([train,loss], feed_dict={X:batch_x,
                                                            T:batch_t,
                                                           dropout_rate:0.3})
        if step % 10 == 0:
            print('Loss : {}'.format(loss_val))
    print('### 학습 끝 ###')

    
    
# Accuracy
predict = tf.argmax(H,1)   # [[0.1 0.3  0.2 0.2 ... 0.1]]

# sklearn을 이용해서 classification_report를 출력해보아요!!

# train데이터로 학습하고 train데이터로 성능평가를 해 보아요!!  

run_train(sess,x_data_train_norm,t_data_train_onehot)

from sklearn.metrics import classification_report
target_name = ['num 0', 'num 1', 'num 2', 'num 3',
               'num 4', 'num 5', 'num 6', 'num 7',
               'num 8', 'num 9']
print(classification_report(t_data_test,
                            sess.run(predict, feed_dict={X:x_data_test_norm,
                                                        dropout_rate:0}),
                            target_names=target_name))


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


label       0
pixel0      0
pixel1      0
pixel2      0
pixel3      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 785, dtype: int64
### 학습 시작 ###
Loss : 0.4554336667060852
Loss : 0.08886462450027466
Loss : 0.05769795551896095
Loss : 0.036862559616565704
Loss : 0.01040507573634386
Loss : 0.03603673726320267
Loss : 0.01870526373386383
Loss : 0.02157888375222683
Loss : 0.003972911741584539
Loss : 0.04204556345939636
### 학습 끝 ###
              precision    recall  f1-score   support

       num 0       0.98      0.99      0.98      1242
       num 1       0.99      0.98      0.99      1429
       num 2       0.97      0.98      0.97      1276
       num 3       0.98      0.96      0.97      1298
       num 4       0.99      0.98      0.98      1236
       num 5       0.97      0.97      0.97      1119
       num 6       0.99      0.99      0.99      1243
       num 7       0.98      0.97      0.98      1334
       num 8       0.96      0.97  